In [ ]:
!pip install -q transformers
!pip install -q torch accelerate
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 8.9 MB/s eta 0:00:00


In [ ]:
import torch
import json
# from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


!hf auth login

model_id = "google/functiongemma-270m-it"
# model_id = "ibm-granite/granite-4.0-tiny-preview"

# Load Model & Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Ofejiro` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate 

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

...

chat_template.jinja:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

In [ ]:
#@title general tool use
# # --- 1. DEFINE THE TOOLS ---

# def get_weather(location: str, unit: str = "celsius") -> str:
#     """
#     Fetches the current weather for a given location.

#     Args:
#         location: The name of the city (e.g., "London", "New York").
#         unit: Temperature unit. Can be 'celsius' or 'fahrenheit'. Defaults to 'celsius'.

#     Returns:
#         A string describing the weather.
#     """
#     print(f"    [System] Calling API for {location}...")
#     if "london" in location.lower():
#         return f"Weather in {location}: Rainy, 15 degrees {unit.capitalize()}."
#     else:
#         return f"Weather in {location}: Sunny, 25 degrees {unit.capitalize()}."

# def convert_currency(amount: float, from_currency: str, to_currency: str) -> str:
#     """
#     Converts an amount from one currency to another.

#     Args:
#         amount: The numerical value to convert.
#         from_currency: The ISO code of the source currency (e.g., "USD").
#         to_currency: The ISO code of the target currency (e.g., "EUR").

#     Returns:
#         A string stating the converted amount and currency.
#     """
#     rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79}
#     converted = amount * (rates.get(to_currency, 1) / rates.get(from_currency, 1))
#     return f"{amount} {from_currency} is approximately {converted:.2f} {to_currency}."

# def convert_length(value: float, from_unit: str, to_unit: str) -> str:
#     """
#     Converts lengths between meters and feet.

#     Args:
#         value: The numeric length to convert.
#         from_unit: The unit of the input value ("meters" or "feet").
#         to_unit: The unit to convert to ("meters" or "feet").

#     Returns:
#         A string showing the conversion result.
#     """
#     factor = 3.28084
#     result = value * factor if from_unit == "meters" else value / factor
#     return f"{value} {from_unit} is equal to {result:.2f} {to_unit}."

# def set_alarm(time_str: str, label: str = "Alarm", repeat: bool = False) -> str:
#     """
#     Sets an alarm for a specific time.

#     Args:
#         time_str: The time for the alarm (e.g., "07:30 AM").
#         label: A descriptive name for the alarm.
#         repeat: Whether the alarm should repeat daily.

#     Returns:
#         A confirmation string of the alarm settings.
#     """
#     return f"Alarm set for {time_str} labeled '{label}'. Repeat: {repeat}."

# def search_wikipedia(query: str, limit: int = 3, language: str = "en") -> list[str]:
#     """
#     Searches Wikipedia for a summary of a topic.

#     Args:
#         query: The search term or topic.
#         limit: The maximum number of results to return. Defaults to 3.
#         language: The 2-letter language code (e.g., "en", "es").

#     Returns:
#         A list of strings containing search result snippets.
#     """
#     return [f"Result {i} for {query} in {language}" for i in range(1, limit + 1)]

# def book_restaurant(restaurant_name: str, party_size: int, time: str, date: str) -> str:
#     """
#     Books a table at a restaurant.

#     Args:
#         restaurant_name: Name of the establishment.
#         party_size: Number of people in the group.
#         time: Time of the reservation (e.g., "19:00").
#         date: Date of the reservation (e.g., "2024-12-25").

#     Returns:
#         A confirmation message with booking details.
#     """
#     return f"Confirmed: Table for {party_size} at {restaurant_name} on {date} at {time}."

# def get_stock_price(ticker: str, exchange: str = "NASDAQ") -> str:
#     """
#     Retrieves the current trading price of a stock.

#     Args:
#         ticker: The stock symbol (e.g., "AAPL", "TSLA").
#         exchange: The stock exchange name. Defaults to "NASDAQ".

#     Returns:
#         A string containing the current stock price.
#     """
#     return f"The current price of {ticker} on {exchange} is $150.25."

# def send_email(recipient: str, subject: str, body: str, cc: list[str] = None) -> str:
#     """
#     Sends an email to a specified recipient.

#     Args:
#         recipient: The email address of the primary receiver.
#         subject: The subject line of the email.
#         body: The main content of the email message.
#         cc: A list of email addresses to carbon copy.

#     Returns:
#         A status message confirming the email was sent.
#     """
#     cc_list = ", ".join(cc) if cc else "None"
#     return f"Email sent to {recipient} (CC: {cc_list}) with subject: {subject}."

# def calculate_mortgage(principal: float, interest_rate: float, years: int, down_payment: float = 0.0) -> dict:
#     """
#     Calculates monthly mortgage payments based on loan terms.

#     Args:
#         principal: The total house price.
#         interest_rate: Annual interest rate as a percentage (e.g., 5.5).
#         years: The term of the loan in years.
#         down_payment: The initial amount paid upfront.

#     Returns:
#         A dictionary containing 'monthly_payment' and 'total_interest'.
#     """
#     return {"monthly_payment": 1200.50, "total_interest": 45000.00}

# def create_playlist(name: str, genres: list[str], mood: str, min_songs: int = 10) -> str:
#     """
#     Creates a music playlist based on genres and mood.

#     Args:
#         name: The name of the new playlist.
#         genres: A list of music genres to include.
#         mood: The target mood (e.g., "chill", "energetic").
#         min_songs: Minimum number of tracks.

#     Returns:
#         A summary string of the created playlist.
#     """
#     return f"Playlist '{name}' created with {min_songs} tracks in {genres} ({mood} mood)."

# def search_flights(origin: str, destination: str, departure_date: str, class_type: str = "economy") -> str:
#     """
#     Searches for available flights between two cities.

#     Args:
#         origin: Departure city airport code or name.
#         destination: Arrival city airport code or name.
#         departure_date: Date of travel (YYYY-MM-DD).
#         class_type: Travel class ("economy", "business", "first").

#     Returns:
#         A string listing the number of flights found.
#     """
#     return f"Found 5 flights from {origin} to {destination} for {departure_date} in {class_type}."

# def update_device_status(room: str, device_type: str, action: str, settings: dict = None) -> str:
#     """
#     Controls smart home devices.

#     Args:
#         room: The room where the device is located.
#         device_type: Type of device (e.g., "Lights", "Thermostat").
#         action: The command (e.g., "on", "off", "set").
#         settings: Optional dictionary for specifics (e.g., {"brightness": 70}).

#     Returns:
#         A status string of the device update.
#     """
#     return f"Action '{action}' performed on {device_type} in {room}. Settings: {settings}."

# def query_database(table: str, columns: list[str], filters: list[dict], limit: int = 10) -> str:
#     """
#     Simulates a SQL-like query structure.

#     Args:
#         table: The name of the database table.
#         columns: List of columns to retrieve.
#         filters: List of dicts with 'column', 'op', and 'value' keys.
#         limit: Maximum number of rows to fetch.

#     Returns:
#         A string representing the generated SQL query.
#     """
#     return f"SELECT {columns} FROM {table} WHERE filters applied LIMIT {limit}."

# def generate_image_prompt(subject: str, style: str, lighting: str, aspect_ratio: str = "16:9") -> str:
#     """
#     Generates a detailed prompt for an AI image generator.

#     Args:
#         subject: The primary focus of the image.
#         style: Artistic style (e.g., "Cyberpunk", "Oil Painting").
#         lighting: Lighting conditions (e.g., "Golden Hour", "Cinematic").
#         aspect_ratio: The dimensions of the image.

#     Returns:
#         A formatted prompt string.
#     """
#     return f"A {style} image of {subject}, {lighting} lighting, {aspect_ratio}."

# def refactor_code(snippet: str, language: str, target_standard: str = "PEP8") -> str:
#     """
#     Refactors a code snippet to meet specific coding standards.

#     Args:
#         snippet: The raw source code string.
#         language: The programming language of the snippet.
#         target_standard: The style guide to follow.

#     Returns:
#         A message confirming the refactoring process.
#     """
#     return f"Refactored {language} code to meet {target_standard} standards."

# def manage_inventory(item_id: str, quantity_change: int, operation: str, warehouse: str = "Main") -> dict:
#     """
#     Updates stock levels in the management system.

#     Args:
#         item_id: The unique identifier for the product.
#         quantity_change: The number of items to add or remove.
#         operation: The type of update ("add", "remove", "audit").
#         warehouse: The storage facility name.

#     Returns:
#         A dictionary with the updated item status and new total.
#     """
#     return {"item_id": item_id, "status": "Updated", "new_total": 150}

# # --- 2. PREPARE TOOLS LIST ---

# tools_list = [
#     get_weather,
#     convert_currency,
#     convert_length,
#     set_alarm,
#     search_wikipedia,
#     book_restaurant,
#     get_stock_price,
#     send_email,
#     calculate_mortgage,
#     create_playlist,
#     search_flights,
#     update_device_status,
#     query_database,
#     generate_image_prompt,
#     refactor_code,
#     manage_inventory
# ]

In [ ]:
%%capture
#@title HOME AUTOMATION TOOLS
# --- HOME AUTOMATION TOOLS ---
def control_light(room: str, state: str, brightness: int = None, color: str = None) -> str:
    """
    Adjusts the state of smart lights in a specific room.

    Args:
        room: The name of the room (e.g., "Living Room", "Kitchen").
        state: Desired state ("on" or "off").
        brightness: Optional brightness level from 0 to 100.
        color: Optional color name or hex code (e.g., "Warm White", "#FF5733").

    Returns:
        A confirmation string of the light adjustment.
    """
    settings = f" to {brightness}%" if brightness else ""
    color_info = f" with color {color}" if color else ""
    return f"Lights in {room} turned {state}{settings}{color_info}."

def set_thermostat(room: str, temperature: float, mode: str = "heat") -> str:
    """
    Sets the target temperature and mode for a room's thermostat.

    Args:
        room: Room name (e.g., "Master Bedroom").
        temperature: Target temperature in Celsius.
        mode: Operation mode ("heat", "cool", "eco", "off").

    Returns:
        A status message from the HVAC system.
    """
    return f"{room} thermostat set to {temperature}°C ({mode} mode)."

def manage_security_system(action: str, code: str) -> str:
    """
    Arms or disarms the home security system.

    Args:
        action: The security action ("arm_away", "arm_home", "disarm").
        code: The 4-digit security PIN.

    Returns:
        A security status report.
    """
    if code == "1234":  # Mock valid code
        return f"Security system: {action.replace('_', ' ').upper()} successfully."
    return "Error: Invalid security code."

def control_smart_blind(room: str, position: int) -> str:
    """
    Sets the position of motorized window blinds.

    Args:
        room: The room name.
        position: Open percentage (0 = Fully Closed, 100 = Fully Open).

    Returns:
        Confirmation of blind position.
    """
    return f"Blinds in {room} moved to {position}% open."

def operate_lock(door_name: str, action: str) -> str:
    """
    Locks or unlocks specific smart bolts.

    Args:
        door_name: Name of the door (e.g., "Front Door", "Garage").
        action: "lock" or "unlock".

    Returns:
        The physical state of the lock.
    """
    return f"{door_name} is now {action}ed."

def get_energy_usage(time_period: str = "today") -> dict:
    """
    Retrieves energy consumption statistics for the home.

    Args:
        time_period: "today", "this_week", or "this_month".

    Returns:
        A dictionary with 'kwh_used' and 'estimated_cost'.
    """
    return {"kwh_used": 12.5, "estimated_cost": "$2.40", "status": "Optimized"}

def broadcast_announcement(message: str, speakers: list[str] = None) -> str:
    """
    Plays a Text-to-Speech message across smart speakers.

    Args:
        message: The text to be announced.
        speakers: List of speaker names. If None, broadcasts to "All".

    Returns:
        Confirmation of the broadcast.
    """
    target = ", ".join(speakers) if speakers else "All speakers"
    return f"Announcement sent to {target}: '{message}'"

def set_media_scene(room: str, device: str, media_title: str, volume: int = 30) -> str:
    """
    Configures an entertainment setup for a specific room.

    Args:
        room: The room name.
        device: Target device ("TV", "Speaker", "Projector").
        media_title: The name of the movie or song.
        volume: Volume level (0-100).

    Returns:
        Confirmation of media playback.
    """
    return f"Playing '{media_title}' on {room} {device} at volume {volume}."

def check_camera_feed(camera_location: str) -> str:
    """
    Retrieves a textual summary of what a smart camera sees.

    Args:
        camera_location: Location (e.g., "Driveway", "Front Porch").

    Returns:
        A description of detected objects or people.
    """
    return f"Camera ({camera_location}): Detected 1 person (recognized: Mail Carrier) and 1 package."

def schedule_vacuum(rooms: list[str], start_time: str) -> str:
    """
    Schedules the robot vacuum for specific rooms.

    Args:
        rooms: List of rooms to clean.
        start_time: Time in 24h format (e.g., "14:30").

    Returns:
        Cleaning schedule confirmation.
    """
    return f"Robot vacuum scheduled to clean {', '.join(rooms)} at {start_time}."

def control_appliance(appliance_name: str, action: str, runtime_minutes: int = None) -> str:
    """
    Controls smart appliances like washers or coffee makers.

    Args:
        appliance_name: e.g., "Coffee Maker", "Dishwasher".
        action: "start", "stop", "pause".
        runtime_minutes: Optional duration for the operation.

    Returns:
        The status of the appliance.
    """
    duration = f" for {runtime_minutes} mins" if runtime_minutes else ""
    return f"{appliance_name} {action}ed{duration}."

def get_sensor_data(sensor_type: str, room: str) -> dict:
    """
    Reads data from environment sensors.

    Args:
        sensor_type: "humidity", "air_quality", "motion", "smoke".
        room: The room name.

    Returns:
        A dictionary containing the 'reading' and 'unit'.
    """
    return {"sensor": sensor_type, "room": room, "reading": 45, "unit": "%"}

def manage_pool(pump_state: str = None, target_temp: float = None, light_on: bool = False) -> str:
    """
    Controls pool maintenance and features.

    Args:
        pump_state: "on" or "off".
        target_temp: Target water temperature.
        light_on: Boolean for pool lights.

    Returns:
        Pool system status summary.
    """
    return f"Pool: Pump {pump_state}, Temp {target_temp}°C, Lights {'On' if light_on else 'Off'}."

def set_smart_plug(alias: str, state: str) -> str:
    """
    Toggles power for a specific smart plug.

    Args:
        alias: The user-defined name of the plug (e.g., "Christmas Tree").
        state: "on" or "off".

    Returns:
        Confirmation of plug state.
    """
    return f"Smart plug '{alias}' is now {state.upper()}."

def trigger_routine(routine_name: str) -> str:
    """
    Executes a pre-defined multi-device routine.

    Args:
        routine_name: e.g., "Good Morning", "Bedtime", "Movie Night", "I'm Home".

    Returns:
        A summary of all actions taken by the routine.
    """
    routines = {
        "Good Morning": "Opening blinds, starting coffee, and reading weather forecast.",
        "Bedtime": "Locking all doors, arming security, and turning off all lights.",
        "Movie Night": "Dimming lights to 10%, closing blinds, and turning on the TV."
    }
    return routines.get(routine_name, f"Routine '{routine_name}' executed.")

# --- TOOL LIST ---
tools_list = [
    control_light, set_thermostat, manage_security_system, control_smart_blind,
    operate_lock, get_energy_usage, broadcast_announcement, set_media_scene,
    check_camera_feed, schedule_vacuum, control_appliance, get_sensor_data,
    manage_pool, set_smart_plug, trigger_routine
]

In [50]:
%%capture
# getting a function detail

import inspect

def get_function_details(func):
    details = {
        "name": func.__name__,
        "args": {},
        "description": inspect.getdoc(func)
    }

    sig = inspect.signature(func)
    for name, param in sig.parameters.items():
        details["args"][name] = {
            "default": param.default if param.default is not inspect._empty else "Required",
            "type": param.annotation.__name__ if hasattr(param.annotation, '__name__') else str(param.annotation)
        }

    return details

# Usage
info = get_function_details(set_smart_plug)
func_info = f"""
Function: {info['name']}\n
Documentation:\n{info['description']}\n
Arguments: {info['args']}
"""

print(func_info)

In [51]:
# --- 3. FIRST TURN: ASK THE QUESTION ---
def start_conversation(user_query: str) -> str:
  user_query = user_query.strip()
  messages = [
      {"role": "user", "content": user_query}
  ]

  # Apply chat template with tools
  input_ids = tokenizer.apply_chat_template(
      messages,
      tools=tools_list,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  # Generate the model's response (It should be a tool call)
  outputs = model.generate(
      input_ids,
      max_new_tokens=128,
      # attention_mask=inputs.attention_mask,
      pad_token_id=tokenizer.eos_token_id,   # Tell it to use EOS as Padding
      )
  generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

  # print(f"--- Step 1: Model Request ---\n{generated_text}\n")
  return generated_text

# ----usage text------
prom = "Make the living room lights a warm sunset orange and set the brightness to 80%."
generated_text = start_conversation(prom)
print(generated_text)


<start_function_call>call:set_smart_plug{alias:<escape>Christmas Tree<escape>,state:<escape>on<escape>}<end_function_call><start_function_call>call:set_smart_plug{state:<escape>on<escape>,alias:<escape>Living Room Lights<escape>}<end_function_call><start_function_call>call:set_brightness(100, 80)<end_function_call>


In [43]:
#@title previous parsing section for single tool usagee
# --- 4. ROBUST PARSING SECTION ---
import re
import json

def parse_tool_call(generated_text: str):
  # 1. Regex to extract the function name and the content inside the curly braces
  # re.DOTALL allows the dot (.) to match newlines, handling multi-line model outputs
  call_pattern = r"call:(\w+)\s*\{(.*?)\}"
  match = re.search(call_pattern, generated_text, re.DOTALL)

  if match:
      func_name = match.group(1)
      raw_args = match.group(2)

      # 2. Extract keys and values using a robust regex
      # Logic:
      # - Group 1 (\w+): Captures the Key (e.g., "location")
      # - Group 2 (.*?): Captures the Value IF it is wrapped in <escape>...</escape> (allows spaces/special chars)
      # - Group 3 ([^,}]+): Captures the Value IF there are no escape tags (stops at comma or closing brace)
      arg_pattern = r"(\w+)\s*:\s*(?:(?:<escape>(.*?)<escape>)|([^,}]+))"

      arg_matches = re.findall(arg_pattern, raw_args, re.DOTALL)
      args_dict = {}

      for key, val_escaped, val_plain in arg_matches:
          # Use the escaped value if present, otherwise use the plain value
          final_val = val_escaped if val_escaped else val_plain
          # Strip surrounding whitespace from the final value
          args_dict[key] = final_val.strip()

      # print(f"Parsed Function: {func_name}")
      # print(f"Parsed Args: {args_dict}")

      # 3. Execute with safety checks
      # Check if the function actually exists in the global scope
      if func_name in globals():
          try:
              # Call the function with the arguments
              observation = globals()[func_name](**args_dict)
              # print(f"Tool Output: {observation}")

              return func_name, args_dict ,observation
          except Exception as e:
              print(f"Error executing function {func_name}: {e}")
              return None, None, None
      else:
          print(f"Error: The function '{func_name}' is not defined.")
          return None, None, None

  else:
      # Handle cases where the model refuses to answer or doesn't use the format
      obs = f"""
            No tool call detected in the model output.
            Raw output was: {generated_text}"
      """

      return None, None, obs

In [52]:
#@title multiple tools parsing algorithm
# --- NEW: Robust Parser (Supports Multiple Calls) ---
def parse_tool_call(generated_text: str):
    # Lists to hold the aggregated results
    tool_calls_list = []
    all_observations = []

    # 1. Regex to extract function name and content inside curly braces
    call_pattern = r"call:(\w+)\s*\{(.*?)\}"
    matches = re.finditer(call_pattern, generated_text, re.DOTALL)

    for match in matches:
        func_name = match.group(1)
        raw_args = match.group(2)

        # 2. Extract keys and values
        arg_pattern = r"(\w+)\s*:\s*(?:(?:<escape>(.*?)<escape>)|([^,}]+))"
        arg_matches = re.findall(arg_pattern, raw_args, re.DOTALL)
        args_dict = {}

        for key, val_escaped, val_plain in arg_matches:
            final_val = val_escaped if val_escaped else val_plain
            args_dict[key] = final_val.strip()

        # Append tool details to list
        tool_calls_list.append({
            "name": func_name,
            "arguments": args_dict
        })

        # 3. Execution
        if func_name in globals():
            try:
                observation = globals()[func_name](**args_dict)
                all_observations.append(observation)
            except Exception as e:
                err = f"Error executing {func_name}: {e}"
                print(err)
                all_observations.append(err)
        else:
            err = f"Error: Function '{func_name}' not defined."
            print(err)
            all_observations.append(err)

    # 4. Return Logic
    if tool_calls_list:
        return tool_calls_list, all_observations
    else:
        obs = f"No tool call detected. Raw output: {generated_text}"
        return [], obs


In [53]:
import inspect
import re
import json
import pandas as pd

In [54]:
#-----sec1------
# total usage
prom = "turn off the living room lights."
# Assuming start_conversation returns the generated text
generated_text = start_conversation(prom)

# UPDATED UNPACKING: We expect lists now
tools_called, tool_results = parse_tool_call(generated_text)

if tools_called:
    for i, tool in enumerate(tools_called):
        print(f"Function: {tool['name']}")
        print(f"Args: {tool['arguments']}")
        print(f"Observation: {tool_results[i]}")
else:
    print("No tools were called.")
    print(f"Reason: {tool_results}")

Function: control_light
Args: {'room': 'Living Room', 'state': 'off'}
Observation: Lights in Living Room turned off.


In [60]:
test_prompts = [
    # Level 1: Direct Mapping
    "Turn on the lights in the living room.",
    "Set the bedroom thermostat to 22 degrees.",
    "Is the front door locked right now?",
    "I'm leaving the house, arm the security system using code 1234.",
    "Open the blinds in the office to 50%.",

    # Level 2: Parameter Extraction
    "Make the kitchen lights a warm sunset orange and set the brightness to 80%.",
    "The air feels a bit dry in the nursery; can you check the humidity levels there?",
    "I need the robot vacuum to clean the hallway and the dining room at 3:00 PM today.",
    "Start the coffee maker for a 10-minute brew cycle.",
    "Tell everyone through the speakers that 'Dinner is ready' but only play it in the playroom and the basement.",

    # Level 3: Contextual Reasoning
    "I'm heading to bed now, make sure everything is secure and the lights are out.",
    "It's getting too bright in here to see my computer screen in the study.",
    "Check if there's anyone standing at the front door.",
    "How much did my electricity cost so far this month?",
    "I'm about to jump in the pool, can you make sure the water is warm and the pump is running?",

    # Level 4: Multi-Step & Complex Logic
    "I want to watch a movie. Dim the living room lights to 10%, close the blinds, and put 'The Matrix' on the TV at volume 40.",
    "If the motion sensor in the garage is triggered, broadcast 'Who is there?' to the garage speakers and lock the inner garage door.",
    "It's a 'Good Morning'—run my morning routine and then set the kitchen temperature to 21 degrees.",
    "The air quality in the living room seems bad. Check the sensor, and if it's over 50, start the air purifier (smart plug) named 'Purifier Alpha'.",
    "I'm coming home early. Unlock the front door, disarm the alarm with code 1234, and turn on the hallway lights."
]

In [61]:
#--------sec3------------
# scoring algorithm

rows_list = []
count = 0

for prompt in test_prompts:
    # 1. Generate Text
    # Assuming start_conversation returns (generated_text, tools_schema)
    generated_text = start_conversation(prompt)

    # 2. Parse Tools (Handles multiple calls)
    tools_called_list, observations_list = parse_tool_call(generated_text)

    # 3. Prepare Data for CSV

    # 'tools_used' is now a list of names for easier reading in CSV
    tools_used_names = [t['name'] for t in tools_called_list]

    # Generate details for the specific tools that were CALLED
    tools_used_details = []
    for tool in tools_called_list:
        func_name = tool['name']
        if func_name in globals():
            # Get the static definition details
            details = get_function_details(globals()[func_name])
            # Optionally add the specific args used in this run
            details['args_used_in_run'] = tool['arguments']
            tools_used_details.append(details)
        else:
            tools_used_details.append({"error": f"Function {func_name} not defined"})

    # If multiple tools were called, observations_list is a list.
    # If none, it's a string error message.
    final_observation_str = str(observations_list)

    event = {
        'prompt': prompt,
        'model_raw_output': generated_text, # The raw text from the model
        'available_tools': tools_list,      # The schema provided to model
        'tools_used': tools_used_names,     # List of function names executed
        'tool_observations': final_observation_str, # The results of execution
        'validation': "", # Placeholder for manual/correct validation
        'model_name': model_id,
        'tools_used_details': tools_used_details
    }

    rows_list.append(event)
    count += 1
    print(f'--------Currently at question {count}')

df = pd.DataFrame(rows_list)
df.to_csv(f'{'functionalgemma'}_tool_usage.csv', index=False)

# total prompt answered
print(f'---------total prompt answered: {count}/{len(test_prompts)}----------')
df.head()

--------Currently at question 1
Error executing set_smart_plug: set_smart_plug() missing 1 required positional argument: 'state'
--------Currently at question 2
--------Currently at question 3
Error executing control_smart_blind: control_smart_blind() got an unexpected keyword argument 'code'
--------Currently at question 4
--------Currently at question 5
--------Currently at question 6
--------Currently at question 7
--------Currently at question 8
Error executing set_smart_plug: set_smart_plug() missing 1 required positional argument: 'state'
Error executing set_smart_plug: set_smart_plug() missing 1 required positional argument: 'state'
Error executing set_media_scene: set_media_scene() missing 1 required positional argument: 'media_title'
--------Currently at question 9
Error executing control_smart_blind: control_smart_blind() got an unexpected keyword argument 'action'
Error executing control_smart_blind: control_smart_blind() got an unexpected keyword argument 'action'
--------C

,prompt,model_raw_output,available_tools,tools_used,tool_observations,validation,model_name,tools_used_details
0,Turn on the lights in the living room.,<start_function_call>call:control_light{room:<...,"[<function control_light at 0x7f82184932e0>, <...",[control_light],['Lights in Living Room turned on.'],,google/functiongemma-270m-it,"[{'name': 'control_light', 'args': {'room': {'..."
1,Set the bedroom thermostat to 22 degrees.,<start_function_call>call:set_smart_plug{alias...,"[<function control_light at 0x7f82184932e0>, <...",[set_smart_plug],"[""Error executing set_smart_plug: set_smart_pl...",,google/functiongemma-270m-it,"[{'name': 'set_smart_plug', 'args': {'alias': ..."
2,Is the front door locked right now?,I cannot assist with locking doors. My current...,"[<function control_light at 0x7f82184932e0>, <...",[],No tool call detected. Raw output: I cannot as...,,google/functiongemma-270m-it,[]
3,"I'm leaving the house, arm the security system...",<start_function_call>call:control_smart_blind{...,"[<function control_light at 0x7f82184932e0>, <...",[control_smart_blind],"[""Error executing control_smart_blind: control...",,google/functiongemma-270m-it,"[{'name': 'control_smart_blind', 'args': {'roo..."
4,Open the blinds in the office to 50%.,<start_function_call>call:set_media_scene{devi...,"[<function control_light at 0x7f82184932e0>, <...",[set_media_scene],"[""Playing 'Movie Night' on Office Speaker at v...",,google/functiongemma-270m-it,"[{'name': 'set_media_scene', 'args': {'room': ..."
